In [1]:
from DbConnector import DbConnector
#from part2 import Database
from tabulate import tabulate

# Set up the program
from dotenv import load_dotenv
import os
import numpy as np
from datetime import datetime, timedelta
from haversine import haversine, Unit
from pprint import pprint

load_dotenv()

connector = DbConnector()

You are connected to the database: assignment3
-----------------------------------------------



### Task 1

How many users, activities and trackpoints are there in the dataset (after it is inserted into the database)

### Task 2
Find the average number of activities per user.

In [65]:
collection = connector.db["Activity"]

rows = collection.aggregate(
    [

        {"$group": {"_id":  "$user_id", "count": {"$sum": 1}}},
        {
        "$group": {
            "_id": None,
            "average avtivites": {"$avg": "$count"}
            }
        },
        {
            "$project": {
                "_id": 0,
                "average_activities": 0
            }
        }

    ]
)

for row in rows:
    print(row)






{'average avtivites': 51.82236842105263}


### Task 3

Find the top 20 users with the highest number of activities.

In [66]:
collection = connector.db["Activity"]

rows = collection.aggregate(
    [
        {"$group": {"_id":  "$user_id", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}},
        {
            "$setWindowFields": {
                "sortBy": {"count": -1},
                "output": {"Top": {"$rank": {}}},
            }
        },
        {"$limit": 20},
    ]
)

for row in rows:
    pprint(row)

{'Top': 1, '_id': '025', 'count': 715}
{'Top': 2, '_id': '128', 'count': 519}
{'Top': 3, '_id': '062', 'count': 406}
{'Top': 4, '_id': '041', 'count': 399}
{'Top': 5, '_id': '004', 'count': 346}
{'Top': 6, '_id': '140', 'count': 345}
{'Top': 7, '_id': '017', 'count': 265}
{'Top': 8, '_id': '003', 'count': 261}
{'Top': 9, '_id': '014', 'count': 236}
{'Top': 10, '_id': '030', 'count': 210}
{'Top': 11, '_id': '011', 'count': 201}
{'Top': 12, '_id': '039', 'count': 198}
{'Top': 13, '_id': '034', 'count': 180}
{'Top': 14, '_id': '000', 'count': 155}
{'Top': 15, '_id': '002', 'count': 146}
{'Top': 16, '_id': '142', 'count': 138}
{'Top': 17, '_id': '037', 'count': 129}
{'Top': 18, '_id': '013', 'count': 119}
{'Top': 19, '_id': '042', 'count': 110}
{'Top': 20, '_id': '020', 'count': 94}


### Task 4
Find all users who have taken a taxi. 

In [67]:
collection = connector.db["Activity"]

rows = collection.aggregate(
    [
        {"$match": {"transportation_mode" : "taxi"}},
        {"$group": {"_id":  "$user_id"}},
        {"$sort": {"_id": 1}},

    ]
)

for row in rows:
    print(row)

{'_id': '010'}
{'_id': '058'}
{'_id': '062'}
{'_id': '078'}
{'_id': '080'}
{'_id': '085'}
{'_id': '098'}
{'_id': '111'}
{'_id': '128'}
{'_id': '163'}


### Task 5
Find all types of transportation modes and count how many activities that are tagged with these transportation mode labels. Do not count the rows where the mode is null.

### Task 6
a) Find the year with the most activities.

b) Is this also the year with most recorded hours?

### Task 7
Find the total distance (in km) walked in 2008, by user with id=112.

### Task 8

 Find the top 20 users who have gained the most altitude meters

In [72]:
collection = connector.db["TrackPoint"]

rows = collection.aggregate(
    [
        {"$match": {"altitude": {"$ne": -777}}},
        {
            "$setWindowFields": {
                "partitionBy": "$activity_id",
                "sortBy": {"date_time": 1},
                "output": {
                    "previous_altitude": {"$shift": {"output": "$altitude", "by": -1}}
                },
            }
        },
        {
            "$project": {
                "activity_id": "$activity_id",
                "difference": {"$subtract": ["$altitude", "$previous_altitude"]},
            }
        },
        {"$match": {"difference": {"$gt": 0}}},
        {
            "$lookup": {
                "from": "Activity",
                "localField": "activity_id",
                "foreignField": "_id",
                "as": "activity",
            }
        },
        {
            "$group": {
                "_id": {
                    "user_id": "$activity.user_id",
                },
                "sum": {"$sum": "$difference"},
            }
        },
        {
            "$project": {
                "altitude_in_meters": {"$multiply": ["$sum", 0.304]},
            }
        },
        {"$sort": {"altitude_in_meters": -1}},
        {
            "$setWindowFields": {
                "sortBy": {"altitude_in_meters": -1},
                "output": {"Top": {"$rank": {}}},
            }
        },
        {"$limit": 20},
    ]
)

for row in rows:
    print(row)

OperationFailure: PlanExecutor error during aggregation :: caused by :: can't $subtract string from string, full error: {'ok': 0.0, 'errmsg': "PlanExecutor error during aggregation :: caused by :: can't $subtract string from string", 'code': 14, 'codeName': 'TypeMismatch'}

### Task 9
Find all users who have invalid activities, and the number of invalid activities per user

In [17]:
collection = connector.db["TrackPoint"]

rows = collection.aggregate(
    [
        {
            "$setWindowFields": {

                "sortBy": {"_id": 1},
                "output": {
                    "previous_activity_id" : {"$shift": {"output": "$activity_id", "by": -1}},
                    "previous_date_time": {"$shift": {"output": "$date_time", "by": -1}}
                },
            }
        },
        {
             "$project": {
                    "_id": "$_id",
                    "activity_id": "$activity_id",
                    "previous_activity_id": "$previous_activity_id",
                    "difference": {"$subtract": [ {"$toDate": "$date_time"}, { "$toDate": "$previous_date_time" } ]},
             }
        },
        {"$match": {"$expr": {"$eq" : [ "$activity_id" , "$previous_activity_id"]}}},
        { "$match": { "difference": { "$gt": 300000 } } },
        {
            "$group": {
                "_id": "$activity_id",
            }
        }





    ]
)

ids = [row["_id"] for row in rows]


collection = connector.db["Activity"]
activities = collection.find({"_id": {"$in": ids}})

users = {}
for activity in activities:

    id = activity["user_id"]
    if id not in users:
        users[id] = {"id": id, "invalid_activities": 1}
    else:
        users[id]["invalid_activities"] += 1

for user in users:
    print(users[user])


{'id': '000', 'invalid_activities': 101}
{'id': '001', 'invalid_activities': 45}
{'id': '002', 'invalid_activities': 98}
{'id': '003', 'invalid_activities': 179}
{'id': '004', 'invalid_activities': 219}
{'id': '005', 'invalid_activities': 44}
{'id': '006', 'invalid_activities': 17}
{'id': '007', 'invalid_activities': 30}
{'id': '008', 'invalid_activities': 16}
{'id': '009', 'invalid_activities': 31}
{'id': '011', 'invalid_activities': 32}
{'id': '012', 'invalid_activities': 43}
{'id': '013', 'invalid_activities': 29}
{'id': '014', 'invalid_activities': 118}
{'id': '015', 'invalid_activities': 46}
{'id': '016', 'invalid_activities': 20}
{'id': '017', 'invalid_activities': 129}
{'id': '018', 'invalid_activities': 27}
{'id': '019', 'invalid_activities': 31}
{'id': '020', 'invalid_activities': 11}
{'id': '021', 'invalid_activities': 1}
{'id': '022', 'invalid_activities': 55}
{'id': '023', 'invalid_activities': 11}
{'id': '024', 'invalid_activities': 27}
{'id': '025', 'invalid_activities': 

In [ ]:
import datetime

collection = connector.db["TrackPoint"]

trackpoints = collection.find()


invalid_activities_ids = []

previous_trackpoint = trackpoints[0]
for trackpoint in trackpoints:

    if trackpoint["activity_id"] not in invalid_activities_ids:

        # Parse from string to date
        current_date = datetime.datetime.strptime(trackpoint["date_time"],'%Y-%m-%d %H:%M:%S')
        previous_date = datetime.datetime.strptime(previous_trackpoint["date_time"],'%Y-%m-%d %H:%M:%S')
        if (current_date - previous_date).total_seconds() / 60 > 5:
            invalid_activities_ids.append(trackpoint["activity_id"])




    previous_trackpoint = trackpoint

collection = connector.db["Activity"]
activities = collection.find({"_id": {"$in": invalid_activities_ids}})

users = {}
for activity in activities:
    id = activity["user_id"]
    if id not in users:
        users[id] = {"id": id, "invalid_activities": 1}
    else:
        users[id]["invalid_activities"] += 1

for user in users:
    print(users[user])

### Task 10
Find the users who have tracked an activity in the Forbidden City of Beijing.

In [32]:
collection = connector.db["TrackPoint"]

rows = collection.aggregate(
    [
        {
         "$project": {
                "activity_id": "$activity_id",
                "distance": {
                    "$sqrt": {
                      "$add": [
                        { "$pow": [{ "$subtract": [{ "$toDouble": "$lat"}, 39.917610]}, 2 ]},
                        { "$pow": [{ "$subtract": [{ "$toDouble": "$lon"}, 116.397028]}, 2 ]}
                        ]
                        }
                    }
                }
        },

        { "$match": { "distance": { "$lt": 0.0045 } }},
        {
            "$group": {
                "_id": "$activity_id",
            }
        }

    ]
)

ids = [row["_id"] for row in rows]

collection = connector.db["Activity"]
activities = collection.find({"_id": {"$in": ids}})

users = {}
for activity in activities:
    id = activity["user_id"]
    if id not in users:
        users[id] = {"user_id": id}

for user in users:
    print(users[user])

{'user_id': '004'}
{'user_id': '018'}
{'user_id': '019'}
{'user_id': '062'}
{'user_id': '067'}
{'user_id': '131'}
{'user_id': '140'}
{'user_id': '168'}


### Task 11
Find all users who have registered transportation_mode and their most used transportation_mode. 

In [ ]:
collection = connector.db["Activity"]

rows = collection.aggregate(
    [
        {"$match": {"transportation_mode": {"$ne": None}}},
        {
            "$group": {
                "_id": {
                    "user_id": "$user_id",
                    "transportation_mode": "$transportation_mode",
                },
                "count": {"$sum": 1},
            }
        },
        {
            "$lookup": {
                "from": "User",
                "let": {
                    "a_id": "$_id",
                    "a_has_labels": "$has_labels",
                },
                "pipeline": [{"$match": {"$expr": {"$eq": ["$$a_id", "$user_id"]}}}],
                "as": "saves_transportation",
            }
        },
        {"$match": {"saves_transportation": {"$ne": {"$a_has_labels": False}}}},
        {"$sort": {"_id.user_id": 1, "count": -1}},
        {
            "$group": {
                "_id": "$_id.user_id",
                "most_used_transportation_mode": {"$first": "$_id.transportation_mode"},
            }
        },
        {"$sort": {"_id": 1}},
    ]
)

for row in rows:
    pprint(row)